In [1]:
# import libriaries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random 
import os
from tqdm import tqdm

from scipy.optimize import minimize
from scipy import optimize

from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# set up the path and import the data
data_folder = "./../../autodl-tmp/data_raw_total/"
auxiliary_folder = "./../../autodl-tmp/"
output_dir = "./../../autodl-tmp/outputs_Ln/"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Directory {output_dir} created.")
else:
    print(f"Directory {output_dir} already exists.")

data_train_AIRS = np.load(f'{data_folder}/data_train_AIRS.npy')
data_train_FGS = np.load(f"{data_folder}/data_train_FGS.npy")

Directory ./../../autodl-tmp/outputs_Ln/ already exists.


In [3]:
# combine two dataset
signal_AIRS_diff_transposed_binned, signal_FGS_diff_transposed_binned  = data_train_AIRS, data_train_FGS

# compress the width of FGS image
FGS_column = signal_FGS_diff_transposed_binned.sum(axis = 2)

# concatenate the two data along the width axis
dataset = np.concatenate([signal_AIRS_diff_transposed_binned, FGS_column[:,:, np.newaxis,:]], axis = 2)

In [4]:
# sum up the pixels along the height axis to transform the data into 2D images (times*wavelengths)
dataset = dataset.sum(axis=3)
dataset.shape

(673, 187, 283)

In [5]:
# search for correct answer
def phase_detector(signal):
    MIN = np.argmin(signal[30:140])+30
    signal1 = signal[:MIN ]
    signal2 = signal[MIN :]

    first_derivative1 = np.gradient(signal1)
    first_derivative1 /= first_derivative1.max()
    first_derivative2 = np.gradient(signal2)
    first_derivative2 /= first_derivative2.max()

    phase1 = np.argmin(first_derivative1)
    phase2 = np.argmax(first_derivative2) + MIN

    return phase1, phase2

def objective(s):
    best_q = 1e10
    for i in range(4) :
        delta = 2
        x = list(range(signal.shape[0]-delta*4))
        y = signal[:p1-delta].tolist() + (signal[p1+delta:p2 - delta] * (1 + s)).tolist() + signal[p2+delta:].tolist()
        
        z = np.polyfit(x, y, deg=i)
        p = np.poly1d(z)
        q = np.abs(p(x) - y).mean()
    
    if q < best_q :
        best_q = q
    
    return q

all_s = []
for i in tqdm(range(len(dataset))):
    
    signal = dataset[i,:,1:].mean(axis=1)
    p1,p2 = phase_detector(signal)
 
    r = minimize(
                objective,
                [0.0001],
                method= 'Nelder-Mead'
                  )
    s = r.x[0]
    all_s.append(s)
    
all_s = np.repeat(np.array(all_s), 283).reshape((len(all_s), 283))        
all_s.shape

100%|██████████| 673/673 [00:33<00:00, 20.31it/s]


(673, 283)

In [6]:
pred = all_s.clip(0)
sigma = np.ones_like(all_s) * 0.000145

In [7]:
# get the targets
train_solution = np.loadtxt(f'{auxiliary_folder}/train_labels.csv', delimiter = ',', skiprows = 1)

targets = train_solution[:,1:]

In [8]:
targets.shape

(673, 283)

In [9]:
# evaluation
rmse = np.sqrt(mean_squared_error(targets, pred))
r2 = r2_score(targets, pred)

print("RMSE:", rmse)
print("R2:", r2)

RMSE: 0.00017095225223725605
R2: 0.9902436058210237
